In [1]:
import os
import math
import random
import numpy as np
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

#### 학습에 필요한 설정값을 정의

In [2]:
# Epsilon 값.
epsilon = 1

# Epsilon 최소값.
epsilonMinimumValue = .001

# 에이전트의 행동 개수(좌로 움직이기, 가만히 있기, 우로 움직이기)
num_actions = 3

# 학습 반복 횟수.
num_epochs = 1000

# 은닉층의 개수.
hidden_size = 128

# Replay Memory 크기.(과거 행위를 기억하기 위한 공간)
maxMemory = 500

# batch size .
batch_size = 50

# 환경 크기.(게임판 크기)
gridSize = 10

# 게임 환경의 현재 상태.(10 x10)
state_size = gridSize * gridSize

# 규제강도.
discount = 0.9

# 학습률.
learning_rage = 0.2

#### 시작과 끝값을 기준으로 랜덤값을 추출하는 함수를 정의

In [3]:
def randf(s, e) :
    return (float(random.randrange(0, (e - s) * 9999)) / 10000) + s

#### DQN 모델을 정의

In [4]:
def build_DQN(x) :
    
    # 표준화를 위한 계수.
    a1 = 1.0 / math.sqrt(float(state_size))
    W1 = tf.Variable(tf.truncated_normal(shape=[state_size, hidden_size], stddev=a1))
    b1 = tf.Variable(tf.truncated_normal(shape=[hidden_size], stddev=0.01))
    H1_output = tf.nn.relu(tf.matmul(x, W1) + b1)
    
    a2 = 1.0 / math.sqrt(float(hidden_size))
    W2 = tf.Variable(tf.truncated_normal(shape=[hidden_size, hidden_size], stddev=a2))
    b2 = tf.Variable(tf.truncated_normal(shape=[hidden_size], stddev=0.01))
    H2_output = tf.nn.relu(tf.matmul(H1_output, W2) + b2)
    
    W3 = tf.Variable(tf.truncated_normal(shape=[hidden_size, num_actions], stddev=a2))
    b3 = tf.Variable(tf.truncated_normal(shape=[num_actions], stddev=0.01))
    output_layer = tf.matmul(H2_output, W3) + b3
    
    return tf.squeeze(output_layer)

In [5]:
# 입력 화면 이미지와 타켓 Q값을 받기 위한 플레이스 홀더(메모리상의 저장공간)를 선언.
x = tf.placeholder(tf.float32, shape=[None, state_size])
y = tf.placeholder(tf.float32, shape=[None, num_actions])

In [6]:
# DQN 모델을 선언하고 예측 결과를 리턴.
y_pred = build_DQN(x)

In [7]:
# MSE 손실 함수와 옵티마이저를 정의.
loss      = tf.reduce_sum(tf.square(y-y_pred)) / (2 * batch_size)
optimizer = tf.train.GradientDescentOptimizer(learning_rage).minimize(loss)

#### 게임 환경 구현

In [8]:
class CatchEnvironment() :
    
    # 상태의 초기값을 지정.
    def __init__(self, gridSize) :
        
        # 그리드 개수.
        self.gridSize = gridSize
        
        # 입력 데이터 개수(그리드 개수 * 그리그 개수)
        self.state_size = self.gridSize * self.gridSize
        
        # 결과를 담을 행렬.
        self.state = np.empty(3, dtype = np.uint8)
        
        
    # 관찰 결과를 리턴.
    def observe(self) :
        # 현재 게임 화면 상태를 받아오기.
        canvas = self.drawState()
        
        # 1차원 행렬로 변환.
        canvas = np.reshape(canvas, (-1, self.state_size))
        
        return canvas
    
    # 게임 현재 상태를 계산.
    def drawState(self) :
        
        # 입력 데이터의 수 만큼의 행렬 계산.
        canvas = np.zeros((self.gridSize, self.gridSize))
        
        # 과일을 설정.
        canvas[self.state[0] - 1, self.state[1] - 1] = 1
        
        # 바구니를 설정.
        canvas[self.gridSize - 1, self.state[2] - 1 - 1] = 1
        canvas[self.gridSize - 1, self.state[2] - 1] = 1
        canvas[self.gridSize - 1, self.state[2] - 1 + 1] = 1
        
        return canvas
    
    # 게임을 초기상태로 리셋.
    def reset(self) :
        
        # 초기 과일 위치 초기화.
        initialFruitColumn    = random.randrange(1, self.gridSize + 1)
        
        # 초기 바구니 위치 초기화.
        initialBucketPosition = random.randrange(2, self.gridSize + 1 - 1)
        
        # 현재 상태 담기.
        self.state            = np.array([1, initialFruitColumn, initialBucketPosition])
        
        return self.getState()
    
    # 현재 상태를 불러오기.
    def getState(self) :
        stateInfo = self.state
        
        # 과일의 세로 위치.
        fruit_row = stateInfo[0]
        # 과일의 가로 위치.
        fruit_col = stateInfo[1]
        # 바구니의 가로 위치.
        basket = stateInfo[2]
        
        return fruit_row, fruit_col, basket
    
    
    # 에이전트가 취한 행동에 대한 보상을 줌.
    def getReward(self) :
        
        # 각 위치값의 위치를 가져오기.
        fruitRow, fruitCol, basket = self.getState()
        
        # 과일이 바닥에 닿았을 경우...
        if (fruitRow == self.gridSize - 1) :
            
            # 바구니가 과일을 받았다면 보상을 1로 반환.
            if (abs(fruitCol - basket) <= 1) :
                return 1
            
            # 과일을 받지 못했다면 보상을 -1로 반환.
            else :
                return -1
            
        # 과일이 바닥에 닿지 않았을 경우...
        else :
            # 아직 바닥에 닿지 않았으므로 보상을 0으로 반환.
            return 0
        
    # 게임이 끝났는지 확인.(1판 종료)
    def isGameOver(self) :
        # 과일이 바닥에 닿았는지 검사.
        if self.state[0] == self.gridSize - 1 :
            return True
        else :
            return False
        
        
    # action(좌, 제자리, 우)에 따라 바구니와 과일의 위치를 수정.
    def updateState(self, action) :
        move = 0
        if action == 0 :
            move = -1
        elif action == 1 :
            move = 0
        elif action == 2 :
            move = 1
            
        # 현재 과일과 바구니 위치를 가져오기.
        fruitRow, fruitCol, basket = self.getState()
        
        # 바구니의 위치를 업데이트.(min, max는 grid 밖으로 벗어나는 것을 방지)
        newBasket = min(max(2, basket + move), self.gridSize - 1)
        
        # 과일은 아래로 한칸 내리기.
        fruitRow = fruitRow + 1
        
        # 현재 상태로 다시 설정.
        self.state = np.array([fruitRow, fruitCol, newBasket])
        
    # 행동 수행.
    def act(self, action) :
        # Action 에 따라 현재 상태를 업데이트.
        self.updateState(action)
        
        # 업데이트된 상태를 보고 보상을 결정.
        reward = self.getReward()
        
        # 현재 게임 한판이 끝났는지 확인.
        gameOver = self.isGameOver()
        
        return self.observe(), reward, gameOver, self.getState()

In [9]:
class ReplayMemory() :
    def __init__(self, gridSize, maxMemory, discount) :
        
        # 초기값 설정.
        
        # 사용할 최대 메모리량.
        self.maxMemory  = maxMemory
        # 게임 환경의 가로 세로 칸의 수.
        self.gridSize   = gridSize
        # 가로 * 세로 칸의 수.
        self.state_size = self.gridSize * self.gridSize
        # 규제 강도.
        self.discount   = discount
        
        # 게임 데이터를 담을 행렬 생성.
        canvas = np.zeros((self.gridSize * self.gridSize))
        canvas = np.reshape(canvas, (-1, self.state_size))
        
        # 입력 데이터를 담을 행렬 생성.
        self.inputState = np.empty((self.maxMemory, 100), dtype = np.float32)
        # 에이전트의 행동 데이터를 담을 행렬.
        self.actions    = np.zeros(self.maxMemory, dtype = np.uint8)
        # 에이전트가 행동을 취한 다음의 게임 상태를 담을 행렬.
        self.nextState  = np.empty((self.maxMemory, 100), dtype = np.float32)
        # 게임 오버 여부.
        self.gameOver   = np.empty(self.maxMemory, dtype = np.bool)
        # 보상.
        self.rewards    = np.empty(self.maxMemory, dtype = np.int8)
        
        # 플레이 횟수.
        self.count   = 0
        # 현재 보상의 결과.
        self.current = 0
        
    # 경험 저장.
    def remember(self, currentState, action, reward, nextState, gameOver) :
        self.actions[self.current]         = action
        self.rewards[self.current]         = reward
        self.inputState[self.current, ...] = currentState
        self.nextState[self.current, ...]  = nextState
        self.gameOver[self.current]        = gameOver
        self.count                         = max(self.count, self.current + 1)
        self.current                       = (self.current + 1) % self.maxMemory
        
    # 입력과 학습을 준비.
    def getBatch(self, y_pred, batch_size, num_actions, state_size, sess, X) :
        # 위에서 설정한 Batch size와 최대 메모리를 비교하여 더 작은 값을 구함.
        memoryLength    = self.count
        chosenBatchSize = min(batch_size, memoryLength)

        # 입력과 결과 데이터를 담을 행렬 생성.
        inputs  = np.zeros((chosenBatchSize, state_size))
        targets = np.zeros((chosenBatchSize, num_actions))
        
         # 배치 안에서 값을 추출하여 담기.
        for i in range(chosenBatchSize) :
            # 배치에 포함될 기억을 랜덤하게 선택.
            randomIndex = random.randrange(0, memoryLength)
            
             # 현재 상태와 Q값을 가져옴.
            current_inputState = np.reshape(self.inputState[randomIndex], (1, 100))
            target             = sess.run(y_pred, feed_dict = {X : current_inputState})

            # 현재 상태 바로 다음 상태를 불러오고 다음 상태에서 취할 수 있는 가장 큰 Q값을 계산.
            current_nextState  = np.reshape(self.nextState[randomIndex], (1, 100))
            nextStateQ         = sess.run(y_pred, feed_dict = {X : current_nextState})
            nextStateMaxQ      = np.amax(nextStateQ)
            
            # 게임 오버일시 보상으로 Q값을 업데이트.
            if (self.gameOver[randomIndex] == True):
                target[self.actions[randomIndex]] = self.rewards[randomIndex]
            else:
                target[self.actions[randomIndex]] = self.rewards[randomIndex] + self.discount * nextStateMaxQ

            # 입력과 결과 데이터에 값을 저장.
            inputs[i]  = current_inputState
            targets[i] = target

        # 결과 리턴.
        return inputs, targets

#### Tensorflow를 통해 학습 가동시 자동으로 호출되는 함수

In [10]:
def main(_) :
    print('학습을 시작하겠습니다.')
    
    # 게임 플레이 환경 선언.
    env = CatchEnvironment(gridSize)
    
    # ReplayMemory 선언.
    memory = ReplayMemory(gridSize, maxMemory, discount)
    
    # 학습된 파라미터를 저장하기 위한 Saver 선언.
    saver = tf.train.Saver()
    
    # 과일을 받은 수.
    winCount = 0
    
    # With 문을 통해 작업이 완료되면 Tensorflow가 자동 종료.
    with tf.Session() as sess :
        # 변수들 초기값 할당.
        sess.run(tf.global_variables_initializer())
        
        # 학습 횟수 만큼 반복.
        for i in range(num_epochs + 1) :
            # 환경 초기화.
            err = 0
            env.reset()
            isGameOver = False
            
            # 최초 상태 가져오기.
            currentState = env.observe()
            
            # 과일이 바닥에 닿을 때까지 반복.
            while isGameOver != True :
                # Q값을 초기화.
                action = -9999
                global epsilon
                
                # 만약 0 ~ 1 사이의 임의값이 앱실론 값보다 작거나 같을 경우...
                if randf(0, 1) <= epsilon :
                    # 랜덤 행동 지정.
                    action = random.randrange(0, num_actions)
                # 클 경우...
                else :
                    # 각 행동에 대한 Q값을 계산.
                    q = sess.run(y_pred, feed_dict = {x : currentState})
                    
                    # q가 가장 큰 행동을 담기.
                    action = q.argmax()
                
                # 앱실론 값에 0.999 값을 곱해서 앱실론 값을 조정.
                if epsilon > epsilonMinimumValue :
                    epsilon = epsilon * 0.999
                
                # 에이전트가 구한 행동을 통해 보상과 다음 상태를 가져옴.
                nextState, reward, gameOver, stateInfo = env.act(action)
                
                # 만약 과일을 받아냈다고 한다면, 점수를 1 증가.
                if reward == 1:
                    winCount = winCount + 1
                    
                # 에이전트가 행동한 결과를 replayMemory에 저장.
                memory.remember(currentState, action, reward, nextState, gameOver)
                
                # 다음 이동을 위해 현재 상태를 재설정.
                currentState = nextState
                
                # 게임 오버 여부를 담기.
                isGameOver = gameOver
                
                # ReplayMemory로 부터 학습에 사용할 Batch 데이터를 가져오기.
                inputs, targets = memory.getBatch(y_pred, batch_size, num_actions, state_size, sess, x)
                
                # 최적화(가중치 수정)을 수행하고, 손실함수를 반환.
                _, loss_print = sess.run([optimizer, loss], feed_dict = {x : inputs, y : targets})
                
                # 손실율 누적.
                err += loss_print
            
            a100 = (float(winCount) / float(i + 1)) * 100
            print(f'반복 : {i}, 에러 : {err}, 승리 : {winCount}, 승리비율 : {a100}')
        
        print('학습이 완료되었습니다.')
        save_path = saver.save(sess, 'model.ckpt')
        print('모델을 저장하였습니다.')

In [11]:
if __name__ == '__main__' :
    # main 함수 호출.
    tf.app.run()

학습을 시작하겠습니다.
반복 : 0, 에러 : 0.0009233761182656508, 승리 : 1, 승리비율 : 100.0
반복 : 1, 에러 : 0.08607495712931268, 승리 : 1, 승리비율 : 50.0
반복 : 2, 에러 : 0.13650890812277794, 승리 : 1, 승리비율 : 33.33333333333333
반복 : 3, 에러 : 0.20879818958928809, 승리 : 1, 승리비율 : 25.0
반복 : 4, 에러 : 0.27168971952050924, 승리 : 1, 승리비율 : 20.0
반복 : 5, 에러 : 0.4092440139502287, 승리 : 1, 승리비율 : 16.666666666666664
반복 : 6, 에러 : 0.404782485216856, 승리 : 2, 승리비율 : 28.57142857142857
반복 : 7, 에러 : 0.38252265378832817, 승리 : 2, 승리비율 : 25.0
반복 : 8, 에러 : 0.4334502089768648, 승리 : 2, 승리비율 : 22.22222222222222
반복 : 9, 에러 : 0.41644234023988247, 승리 : 2, 승리비율 : 20.0
반복 : 10, 에러 : 0.3414793200790882, 승리 : 2, 승리비율 : 18.181818181818183
반복 : 11, 에러 : 0.5242220982909203, 승리 : 2, 승리비율 : 16.666666666666664
반복 : 12, 에러 : 0.43035666085779667, 승리 : 2, 승리비율 : 15.384615384615385
반복 : 13, 에러 : 0.3921505808830261, 승리 : 3, 승리비율 : 21.428571428571427
반복 : 14, 에러 : 0.4491836465895176, 승리 : 3, 승리비율 : 20.0
반복 : 15, 에러 : 0.42273159325122833, 승리 : 4, 승리비율 : 25.0
반복 : 16, 에러 :

반복 : 123, 에러 : 0.12994266208261251, 승리 : 32, 승리비율 : 25.806451612903224
반복 : 124, 에러 : 0.15774843376129866, 승리 : 32, 승리비율 : 25.6
반복 : 125, 에러 : 0.16206358838826418, 승리 : 32, 승리비율 : 25.396825396825395
반복 : 126, 에러 : 0.18058714736253023, 승리 : 33, 승리비율 : 25.984251968503933
반복 : 127, 에러 : 0.26796148903667927, 승리 : 33, 승리비율 : 25.78125
반복 : 128, 에러 : 0.20548114459961653, 승리 : 33, 승리비율 : 25.581395348837212
반복 : 129, 에러 : 0.16210596729069948, 승리 : 34, 승리비율 : 26.153846153846157
반복 : 130, 에러 : 0.22329303435981274, 승리 : 35, 승리비율 : 26.717557251908396
반복 : 131, 에러 : 0.13843524549156427, 승리 : 35, 승리비율 : 26.515151515151516
반복 : 132, 에러 : 0.12448479514569044, 승리 : 36, 승리비율 : 27.06766917293233
반복 : 133, 에러 : 0.15538817830383778, 승리 : 36, 승리비율 : 26.865671641791046
반복 : 134, 에러 : 0.14008867926895618, 승리 : 37, 승리비율 : 27.40740740740741
반복 : 135, 에러 : 0.15112204663455486, 승리 : 37, 승리비율 : 27.205882352941174
반복 : 136, 에러 : 0.14091580640524626, 승리 : 37, 승리비율 : 27.00729927007299
반복 : 137, 에러 : 0.1276488811708986

반복 : 241, 에러 : 0.10915394779294729, 승리 : 87, 승리비율 : 35.9504132231405
반복 : 242, 에러 : 0.11315216217190027, 승리 : 87, 승리비율 : 35.80246913580247
반복 : 243, 에러 : 0.12434917967766523, 승리 : 88, 승리비율 : 36.0655737704918
반복 : 244, 에러 : 0.12946113757789135, 승리 : 89, 승리비율 : 36.3265306122449
반복 : 245, 에러 : 0.11410030722618103, 승리 : 89, 승리비율 : 36.17886178861789
반복 : 246, 에러 : 0.13811397459357977, 승리 : 89, 승리비율 : 36.032388663967616
반복 : 247, 에러 : 0.10493575781583786, 승리 : 90, 승리비율 : 36.29032258064516
반복 : 248, 에러 : 0.08275373233482242, 승리 : 90, 승리비율 : 36.144578313253014
반복 : 249, 에러 : 0.13219287991523743, 승리 : 91, 승리비율 : 36.4
반복 : 250, 에러 : 0.09112114878371358, 승리 : 91, 승리비율 : 36.254980079681275
반복 : 251, 에러 : 0.10616053408011794, 승리 : 91, 승리비율 : 36.11111111111111
반복 : 252, 에러 : 0.17199210450053215, 승리 : 92, 승리비율 : 36.36363636363637
반복 : 253, 에러 : 0.09945899620652199, 승리 : 92, 승리비율 : 36.22047244094488
반복 : 254, 에러 : 0.11065896973013878, 승리 : 92, 승리비율 : 36.07843137254902
반복 : 255, 에러 : 0.0746559691615402

반복 : 358, 에러 : 0.033852706430479884, 승리 : 168, 승리비율 : 46.796657381615596
반복 : 359, 에러 : 0.03214091691188514, 승리 : 169, 승리비율 : 46.94444444444444
반복 : 360, 에러 : 0.03315580519847572, 승리 : 170, 승리비율 : 47.091412742382275
반복 : 361, 에러 : 0.03192237741313875, 승리 : 171, 승리비율 : 47.23756906077348
반복 : 362, 에러 : 0.027351397089660168, 승리 : 172, 승리비율 : 47.38292011019284
반복 : 363, 에러 : 0.03326626494526863, 승리 : 173, 승리비율 : 47.527472527472526
반복 : 364, 에러 : 0.0239550843834877, 승리 : 174, 승리비율 : 47.671232876712324
반복 : 365, 에러 : 0.04353367444127798, 승리 : 175, 승리비율 : 47.81420765027322
반복 : 366, 에러 : 0.031116751953959465, 승리 : 176, 승리비율 : 47.956403269754766
반복 : 367, 에러 : 0.023239271598868072, 승리 : 177, 승리비율 : 48.09782608695652
반복 : 368, 에러 : 0.024861737270839512, 승리 : 178, 승리비율 : 48.23848238482385
반복 : 369, 에러 : 0.0320166009478271, 승리 : 179, 승리비율 : 48.37837837837838
반복 : 370, 에러 : 0.031004087533801794, 승리 : 180, 승리비율 : 48.517520215633425
반복 : 371, 에러 : 0.02574370475485921, 승리 : 181, 승리비율 : 48.65591397849

반복 : 473, 에러 : 0.05674848554190248, 승리 : 269, 승리비율 : 56.75105485232067
반복 : 474, 에러 : 0.11952316761016846, 승리 : 270, 승리비율 : 56.84210526315789
반복 : 475, 에러 : 0.09061987069435418, 승리 : 271, 승리비율 : 56.9327731092437
반복 : 476, 에러 : 0.023854987928643823, 승리 : 271, 승리비율 : 56.81341719077568
반복 : 477, 에러 : 0.07931541337165982, 승리 : 272, 승리비율 : 56.903765690376574
반복 : 478, 에러 : 0.0830461559817195, 승리 : 272, 승리비율 : 56.78496868475992
반복 : 479, 에러 : 0.02781190408859402, 승리 : 273, 승리비율 : 56.875
반복 : 480, 에러 : 0.10496981185860932, 승리 : 274, 승리비율 : 56.96465696465697
반복 : 481, 에러 : 0.12148437486030161, 승리 : 275, 승리비율 : 57.05394190871369
반복 : 482, 에러 : 0.04428668518085033, 승리 : 276, 승리비율 : 57.14285714285714
반복 : 483, 에러 : 0.08861061255447567, 승리 : 277, 승리비율 : 57.231404958677686
반복 : 484, 에러 : 0.05050772521644831, 승리 : 278, 승리비율 : 57.319587628865975
반복 : 485, 에러 : 0.05789124651346356, 승리 : 279, 승리비율 : 57.407407407407405
반복 : 486, 에러 : 0.063020427711308, 승리 : 280, 승리비율 : 57.49486652977412
반복 : 487, 에러 : 0

...

반복 : 703, 에러 : 0.0029698458092752844, 승리 : 485, 승리비율 : 68.89204545454545
반복 : 704, 에러 : 0.0024626953236293048, 승리 : 486, 승리비율 : 68.93617021276596
반복 : 705, 에러 : 0.002101282778312452, 승리 : 487, 승리비율 : 68.98016997167139
반복 : 706, 에러 : 0.002015426551224664, 승리 : 488, 승리비율 : 69.02404526166902
반복 : 707, 에러 : 0.0019267030656919815, 승리 : 489, 승리비율 : 69.0677966101695
반복 : 708, 에러 : 0.002158801828045398, 승리 : 490, 승리비율 : 69.1114245416079
반복 : 709, 에러 : 0.0018789213645504788, 승리 : 491, 승리비율 : 69.15492957746478
반복 : 710, 에러 : 0.002064383697870653, 승리 : 492, 승리비율 : 69.19831223628692
반복 : 711, 에러 : 0.001846442959504202, 승리 : 493, 승리비율 : 69.24157303370787
반복 : 712, 에러 : 0.0017379541095579043, 승리 : 494, 승리비율 : 69.28471248246845
반복 : 713, 에러 : 0.0015636518728570081, 승리 : 495, 승리비율 : 69.32773109243698
반복 : 714, 에러 : 0.0028243677952559665, 승리 : 496, 승리비율 : 69.37062937062936
반복 : 715, 에러 : 0.0025708753237267956, 승리 : 497, 승리비율 : 69.41340782122904
반복 : 716, 에러 : 0.0017349373520119116, 승리 : 498, 승리비율 : 69.

반복 : 817, 에러 : 0.0009214284218614921, 승리 : 599, 승리비율 : 73.22738386308069
반복 : 818, 에러 : 0.0007642562850378454, 승리 : 600, 승리비율 : 73.26007326007326
반복 : 819, 에러 : 0.0005008335538150277, 승리 : 601, 승리비율 : 73.29268292682927
반복 : 820, 에러 : 0.0006944535161892418, 승리 : 602, 승리비율 : 73.32521315468941
반복 : 821, 에러 : 0.0005940521223237738, 승리 : 603, 승리비율 : 73.35766423357664
반복 : 822, 에러 : 0.0006467922212323174, 승리 : 604, 승리비율 : 73.39003645200486
반복 : 823, 에러 : 0.00037326340134313796, 승리 : 605, 승리비율 : 73.42233009708737
반복 : 824, 에러 : 0.0006449993379646912, 승리 : 606, 승리비율 : 73.45454545454545
반복 : 825, 에러 : 0.0004864584461756749, 승리 : 607, 승리비율 : 73.48668280871671
반복 : 826, 에러 : 0.0004605051526596071, 승리 : 608, 승리비율 : 73.51874244256348
반복 : 827, 에러 : 0.0006464214275183622, 승리 : 609, 승리비율 : 73.55072463768117
반복 : 828, 에러 : 0.0004814362655451987, 승리 : 610, 승리비율 : 73.58262967430639
반복 : 829, 에러 : 0.00038796630906290375, 승리 : 611, 승리비율 : 73.6144578313253
반복 : 830, 에러 : 0.0005049421615694882, 승리 : 612, 승리

반복 : 930, 에러 : 0.00038621789644821547, 승리 : 712, 승리비율 : 76.47690655209452
반복 : 931, 에러 : 0.0004083129606442526, 승리 : 713, 승리비율 : 76.50214592274678
반복 : 932, 에러 : 0.0005979096113151172, 승리 : 714, 승리비율 : 76.52733118971061
반복 : 933, 에러 : 0.00048564260578132235, 승리 : 715, 승리비율 : 76.55246252676659
반복 : 934, 에러 : 0.00037064534990349784, 승리 : 716, 승리비율 : 76.57754010695187
반복 : 935, 에러 : 0.00029051645105937496, 승리 : 717, 승리비율 : 76.6025641025641
반복 : 936, 에러 : 0.00031238542123901425, 승리 : 718, 승리비율 : 76.62753468516541
반복 : 937, 에러 : 0.00046906989518902265, 승리 : 719, 승리비율 : 76.65245202558634
반복 : 938, 에러 : 0.000255259481491521, 승리 : 720, 승리비율 : 76.6773162939297
반복 : 939, 에러 : 0.0003758864040719345, 승리 : 721, 승리비율 : 76.70212765957447
반복 : 940, 에러 : 0.00043375913446652703, 승리 : 722, 승리비율 : 76.7268862911796
반복 : 941, 에러 : 0.00028698185360553907, 승리 : 723, 승리비율 : 76.75159235668791
반복 : 942, 에러 : 0.00036410749180504354, 승리 : 724, 승리비율 : 76.7762460233298
반복 : 943, 에러 : 0.0003098036722803954, 승리 : 725,

SystemExit: 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
